In [118]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.base import clone
from sklearn.model_selection import train_test_split
import pandas as pd

### Implementation of greedy forward feature selection for instance space analysis
Algorithm:
- Start with empty feature set
- Evaluate performance of model using each feature individually
- Add the feature which results in the greatest improvement in performance to current set of features
- Repeat previous 2 steps until stopping criterion is met (performance improvement drops below some threshold, delta performance is no longer statitstically significant)


In [79]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 123)

In [32]:
knn = KNeighborsClassifier(n_neighbors=3) # estimator
sfs = SequentialFeatureSelector(knn, n_features_to_select=3)
sfs.fit(X,y)
subset = sfs.transform(X)
sfs.support_

array([ True, False,  True,  True])

In [134]:
class GreedyForwardSelection():

    def __init__(self, k_features):
        self.k_features = k_features # target number of features, termination conditon
        #self.model = clone(model)

    def fit(self, X_train, X_test, y_train, y_test):
        max_idx = tuple(range(X_train.shape[1])) # get feature idxs
        total_features_count = len(max_idx) # initial num of features
        self.subsets_ = list()
        self.scores_ = list()
        self.indices_ = list()

        scores = list()
        subsets = list()
        
        for p in combinations(max_idx, r=1):
            # train on all possible first features
            score = self._calc_score(X_train, X_test, y_train, y_test, p)
            scores.append(score)
            subsets.append(p)

        best_score_idx = np.argmax(scores)
        self.scores_.append(scores[best_score_idx])
        self.indices_ = list(subsets[best_score_idx])
        self.subsets_.append(self.indices_)

        # add features sequentially until k_features is reached
        current_k = 1
        while current_k < self.k_features:
            scores = list()
            subsets = list()

            idx = 0
            while idx < total_features_count:
                if idx not in self.indices_:
                    indices = list(self.indices_)
                    indices.append(idx)
                    score = self._calc_score(X_train, X_test, y_train, y_test, indices)
                    scores.append(score)
                    subsets.append(indices)
                idx += 1
            
            best_score_idx = np.argmax(scores)
            self.scores_.append(scores[best_score_idx])
            self.indices_ = list(subsets[best_score_idx])
            self.subsets_.append(self.indices_)
            current_k += 1
        
        self.k_score_ = self.scores_[-1]

    # output data matrix using reduced subset of features
    def transform(self, X):
        return X[:, self.indices_]

    ##def _calc_score(self, X_train, X_test, y_train, y_test, indices):
       # self.model.fit(X_train[:, indices], y_train)
       # score = self.model.score(X_test[:, indices], y_test)
       # return score

    # add custom calc score function for pca
    def _calc_score(self, X_train, X_test, y_train, y_test, indices):
        epsilon_range = np.linspace(0.05, 2, 100)
        components = len(indices)
        accuracies = list()

        # train model
        pca_fit = PCA(n_components=components).fit(X_train[:, indices])
        pca_train = pca_fit.transform(X_train[:, indices])

        # project test instances
        pca_test = pca_fit.transform(X_test[:, indices])

        for epsilon in epsilon_range:
            predictions = list()
            for test_instance in pca_test:
                neighbours_idx = np.argwhere(np.linalg.norm(pca_train - test_instance, axis=1) < epsilon)

                # if no neighbouring points in epsilon radius, use 1-NN
                if len(neighbours_idx) == 0:
                    distances = np.linalg.norm(pca_train - test_instance, axis=1)
                    neighbours_idx = np.argmin(distances)
                
                neighbour_pts = y_train[neighbours_idx]
                unique, counts = np.unique(neighbour_pts, return_counts=True)
                pred = unique[np.argmax(counts)]
                predictions.append(pred)
            matches = np.count_nonzero(predictions == y_test)
            acc = matches / len(y_test)
            accuracies.append(acc)
        
        best_score = np.max(accuracies) # return max acc achieved across all epsilons
        return best_score